In [1]:
    import torch.nn as nn
    import torch
    import numpy as np
    import gym

    import ipynb.fs.full.misc as misc
    import ipynb.fs.full.breeder as br
    import ipynb.fs.full.mutation_functions as mf
    import ipynb.fs.full.selection_functions as sf
    import ipynb.fs.full.crossover_functions as cf
    from ipynb.fs.full.agent_class import Agent
    import ipynb.fs.full.net_class as nm

In [2]:
    class CartPoleAgentNet(nm.NetModule):
        def __init__(self):
            super(CartPoleAgentNet, self).__init__()

            self.num_inputs  = 4
            self.num_outputs = 2

            #  define net structure here
            self.model = nn.Sequential(
                nn.Linear(self.num_inputs, 32),
                nn.Tanh(),            
                nn.Linear(32, 32),
                nn.Tanh(),
                nn.Linear(32, self.num_outputs),
                nn.Tanh(),
            )

            self.model = self.model.cuda()

            self.parameters_count = self.count_parameters()

        def forward(self, inputs):
            return self.model(inputs)

    class CartPoleAgent(Agent):
        def __init__(self, dna=None):
            super(CartPoleAgent, self).__init__(dna, CartPoleAgentNet())

        def step(self, observation):
            state = np.asarray(observation)

            torch_state = misc.single_to_torch(state, cuda=self.net.is_cuda())
            torch_action = self.net(torch_state)
            action = misc.single_from_torch(torch_action, cuda=self.net.is_cuda())

            self.add_experience(state, action)

            return np.argmax(action)

        @staticmethod
        def produce_offspring(dna):
            return CartPoleAgent(dna=dna)

    misc.init_random(1337)

    env = gym.make("CartPole-v0")

    population_size = 50
    population = [CartPoleAgent()] * population_size

    breeder = br.Breeder(population,
                         env,
                         85.,
                         generations=200,
                         elite_percent=0.1,
                         mutation_rate=0.9,
                         selection_pressure=0.2,
                         selection_func=sf.tournament,
                         crossover_func=cf.none,
                         mutation_func=mf.gaussian,
                         render=False)

    agent = breeder.evolve()


    for _ in range(10):
        agent.evaluate(env, render=True, sleep=True)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Gen: 0 , Max fitness: 39.70335535985003 , Min fitness: 39.70335535985003 , Age: 0 , Mean fitness: 39.70335535985005
en 5
Gen: 1 , Max fitness: 42.60948095476121 , Min fitness: 7.64827525163591 , Age: 0 , Mean fitness: 18.30027458667165
en 5
Gen: 2 , Max fitness: 53.79563497335242 , Min fitness: 7.64827525163591 , Age: 1 , Mean fitness: 18.097053022337533
en 5
Gen: 3 , Max fitness: 58.931534704845895 , Min fitness: 7.64827525163591 , Age: 2 , Mean fitness: 19.687228357648646
en 5
Gen: 4 , Max fitness: 63.839079344197714 , Min fitness: 7.64827525163591 , Age: 0 , Mean fitness: 27.510115844041056
en 5
Gen: 5 , Max fitness: 69.65772784747834 , Min fitness: 7.64827525163591 , Age: 1 , Mean fitness: 24.197281031555658
en 5
Gen: 6 , Max fitness: 67.83389185508568 , Min fitness: 7.64827525163591 , Age: 0 , Mean fitness: 27.58840925116353
en 5
Gen: 7 , Max fitness: 75.24074452117061 , Min fitness: